<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/harry_potter/HarryPotterNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install dependencies
!pip install selenium neo4j spacy==2.3.2 neo4j

# Setup selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

# Setup SpaCy and NeuralCoref
from google.colab import drive
drive.mount('/content/gdrive')

!git clone https://github.com/huggingface/neuralcoref.git
!pip install -U 
!python -m spacy download en_core_web_sm

%cd neuralcoref

!pip install -r requirements.txt
!pip install -e .

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [62]:
def enrich_single_item(item, index):
  global wd
  try:
    div = wd.find_element_by_xpath(f"//div[@data-source = '{item}']")
    try:
      result = find_element_by_name("a").text
    except:
      result = find_element_by_name("div").text
    character_dict[chapter][index][item] = result
  except:
    pass
  

In [71]:
import time

c1characters = "https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Philosopher%27s_Stone_(character_index)"

def get_characters(url):
  # Get the list of characters by chapter
  wd.get(url)
  character_dict = dict()
  elem = wd.find_element_by_class_name("mw-parser-output")
  
  # Locate character by chapter
  tables = elem.find_elements_by_tag_name('table')
  for i, chapter in enumerate(tables):
    list_of_characters = []
    characters = chapter.find_elements_by_tag_name('a')
    for character in characters:
      if not character.get_attribute('title'):
        continue
      list_of_characters.append({'title': character.get_attribute('title'), 'url': character.get_attribute('href')})
    character_dict['chapter_' + str(i + 1)] = list_of_characters
  
  # Enrich characters with additional information
  for chapter in character_dict:
    for index, character in enumerate(character_dict[chapter]):
      # Get the character page with selenium
      wd.get(character['url'])
      # Enrich aliases
      try:
        alias_div = wd.find_element_by_xpath("//div[@data-source = 'alias']")
        aliases = alias_div.find_elements_by_tag_name('li')
        result = []
        for a in aliases:
          # Ignore under the cloak-guise and the name he told
          if "disguise" in a.text or "the name he told" in a.text:
            continue
          alias = a.text.split('[')[0].split('(')[0].strip()
          result.append(alias)
        character_dict[chapter][index]['aliases'] = result
      except:
        pass
      # Enrich loyalties
      try:
        loyalty_div = wd.find_element_by_xpath("//div[@data-source = 'loyalty']")
        loyalties = loyalty_div.find_elements_by_tag_name('li')
        result = []
        for l in loyalties:
          # Ignore under the cloak-guise and the name he told
          loyalty = l.text.split('[')[0].split('(')[0].strip()
          result.append(loyalty)
        character_dict[chapter][index]['loyalty'] = result
      except:
        pass
      # Enrich family relationships
      try:
        family_div = wd.find_element_by_xpath("//div[@data-source = 'loyalty']")
        relationships = loyalty_div.find_elements_by_tag_name('li')
        result = []
        for r in relationships:
          # Ignore under the cloak-guise and the name he told
          rel = l.text.split('[')[0].split('(')[0].strip()
          rel_type = l.text.split('(')[-1][:-1]
          result.append({'person':rel, 'type': rel_type})
        character_dict[chapter][index]['family'] = result
      except:
        pass
      # Enrich blood
      enrich_single_item('blood', index)
      # Enrich nationality
      enrich_single_item('nationality', index)
      # Enrich marital status
      enrich_single_item('marital', index)
      # Enrich species
      enrich_single_item('species', index)
      # Enrich house
      enrich_single_item('house', index)
      # Enrich gender
      enrich_single_item('gender', index)




    

In [72]:
get_characters("https://harrypotter.fandom.com/wiki/Harry_Potter_and_the_Philosopher%27s_Stone_(character_index)")

MaxRetryError: ignored

In [5]:
import requests

In [6]:
master = "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"
req = requests.get(master)
req = req.text

In [7]:
import spacy
from spacy.matcher import Matcher
import neuralcoref

nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

doc1 = nlp('My sister has a dog. She loves him.')
print(doc1._.coref_clusters)

[My sister: [My sister, She], a dog: [a dog, him]]


In [8]:
def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

In [9]:
def get_character_dict(chapter):
  super_list = list()
  dicts = [character_dict['chapter_' + str(i)] for i in range(1,chapter + 1)]
  for d in dicts:
    for item in d:
      super_list.append(item)
  return super_list

In [56]:
import sys
import random

hardcoded_options = dict()
hardcoded_options['Malfoy'] = ['Draco Malfoy']
hardcoded_options['Patil'] = ['Padma Patil', 'Parvati Patil']
hardcoded_options['Tom'] = ['Tom']



def handle_multiple_options(result, start, doc):
  needs_deduplication = [(i,x) for i,x in enumerate(result) if len(x['string_id']) > 1 and x['text'] != 'Dursley']
  for index, multiple_options in needs_deduplication:
    # Special logic for Dursleys, if there if Mr. then Vernon, if Mrs. then Petunia
    prefix = doc[start-3:start]
    if (multiple_options['text'] == 'Dursley') and ("Mr." in prefix.text):
      resolution = ["Vernon Dursley"]
    elif (multiple_options['text'] == 'Dursley') and ("Mrs." in prefix.text):
      resolution = ["Petunia Dursley"]
    # Find nearest entity
    else:
      end_char = multiple_options['end']
      distance = sys.maxsize
      resolution = []
      for possible_option in result:
        # Skip multiple options and entities that don't have any of the multiple option
        if (not len(possible_option['string_id']) == 1) or (not possible_option['string_id'][0] in multiple_options['string_id']):
          continue
        new_distance = abs(multiple_options['end'] - possible_option['end'])
        if new_distance < distance:
          distance = new_distance
          resolution = possible_option['string_id']
      
      if not resolution:
        try:
          ho = hardcoded_options[multiple_options['text']]
          if len(ho) == 1:
            resolution = ho
          else:
            resolution = [random.choice(ho)]
        except:
          print(f"no way to disambiguate {multiple_options['text']} from options: {multiple_options['string_id']}")
      result[index]['string_id'] = resolution
  print([el for el in result if len(el['string_id']) > 1])
  return result

In [12]:
from collections import Counter

def get_distances(result):
  #sort by start character
  result = sorted(result, key=lambda k: k['start'])
  compact_entities = []
  for entity in result:
    # If the same entity occurs, prolong the end 
    if (len(compact_entities) > 0) and (compact_entities[-1]['string_id'] == entity['string_id']):
      compact_entities[-1]['end'] = entity['end']
    else:
      compact_entities.append(entity)
  distances = list()
  # We can skip the last entity
  for index, source in enumerate(compact_entities[:-1]):
    for target in compact_entities[index + 1:]:
      if (source['string_id'] != target['string_id']) and (abs(source['end'] - target['start']) < 20):
        try:
          link = sorted([source['string_id'][0], target['string_id'][0]])
        except:
          print(source['string_id'], target['string_id'])
        distances.append(link)
      else:
        break
  return Counter(map(tuple, distances))
  



In [54]:
chapters = req.split("CHAPTER")[1:]
stop_words = ['of', 'the', 'at', 'family', 'keeper', 'wizard', 'fat', 'de', 'hogwarts']

def get_characters_in_chapter(chapter):
  c = chapters[chapter - 1]
  # Prepare characters matcher
  matcher = Matcher(nlp.vocab)
  for character in get_character_dict(chapter):
    names = [el for el in character['title'].split(' ') if len(el) > 2]
    matcher_pattern = []
    matcher_pattern.append([{"LOWER": n.lower(), "IS_TITLE": True} for n in names])

    if not "'" in character['title']:
      for n in names:
        if n.lower() in stop_words:
          continue
        matcher_pattern.append([{"LOWER": n.lower(), "IS_TITLE": True}])
        # Special case for Ronald Weasley -> Also add Ron
        if n == "Ronald":
          matcher_pattern.append([{"LOWER": "ron", "IS_TITLE": True}])

    matcher.add(character['title'], matcher_pattern)

  # Prepare text
  lines = c.split('\n')[1:]
  lines = list(filter(None, lines))
  chapter_title = lines[0]
  print(chapter_title)
  text = " ".join(lines[1:])
  text = coref_resolution(text)

  # Find matches
  doc = nlp(text)
  matches = matcher(doc)
  result = []
  for match_id, start, end in matches:
      string_id = nlp.vocab.strings[match_id]  # Get string representation
      span = doc[start:end]  # The matched span
      exists_longer = [(start == e['start'] and end < e['end']) or (start > e['start'] and end == e['end']) for e in result]
      same = [start == e['start'] and end == e['end'] for e in result]
      shorter_end = [start == e['start'] and end > e['end'] for e in result]
      shorter_start = [start < e['start'] and end == e['end'] for e in result]
      # If there is a longer version already skip
      if any(exists_longer):
        continue
      # If it exists but has a longer end
      if any(shorter_end):
        del result[shorter_end.index(True)]
        result.append({'string_id': [string_id], 'start': start, 'end': end, 'text': span.text})
      #  
      elif any(shorter_start):
        del result[shorter_start.index(True)]
        result.append =({'string_id': [string_id], 'start': start, 'end': end, 'text': span.text})
      # If not exists 
      elif not any(same):
        result.append({'string_id': [string_id], 'start': start, 'end': end, 'text': span.text})
      else:
        i = same.index(True)
        # Add more entities to a single span
        result[i]['string_id'].append(string_id)
  handle_multiple_options(result, start, doc)
  return result




In [ ]:
from neo4j import GraphDatabase
host = 'bolt://3.239.224.171:7687'
user = 'neo4j'
password = 'rainbow-mittens-track'
driver = GraphDatabase.driver(host,auth=(user, password))

def store_to_neo4j(distances):
  data = [{'source': el[0], 'target': el[1], 'weight': distances[el]} for el in distances]
  with driver.session() as session:
    session.run("""
    UNWIND $data as row
    MERGE (c:Character{name:row.source})
    MERGE (t:Character{name:row.target})
    MERGE (c)-[i:INTERACTS]-(t)
    SET i.weight = coalesce(i.weight,0) + row.weight
    """, {'data': data})

In [57]:
for c in range(1,18):
  end = get_characters_in_chapter(c)
  distances = get_distances(end)
  store_to_neo4j(distances)

THE BOY WHO LIVED
[{'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 3, 'end': 4, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 20, 'end': 21, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 40, 'end': 41, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 68, 'end': 69, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 85, 'end': 86, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 100, 'end': 101, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 114, 'end': 115, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudley Dursley'], 'start': 123, 'end': 124, 'text': 'Dursley'}, {'string_id': ['Vernon Dursley', 'Petunia Dursley', 'Dudl